In [62]:
from multiprocessing.dummy import Pool, Queue
import lxml
import tldextract
from tqdm import tqdm
from time import sleep
from bs4 import BeautifulSoup
from bs4.element import Comment
import gzip
import pandas as pd
import codecs
import sys
import os
import json
import re
import numpy as np
from os import listdir
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import GermanStemmer, EnglishStemmer, RussianStemmer
from sklearn.feature_extraction.text import CountVectorizer
import re

In [194]:
regex_symbols = re.compile(r'[^a-zA-Z0-9а-яА-Я\u00E4\u00F6\u00FC\u00C4\u00D6\u00DC\u00df]')
stopwords_new = ['http', 'url', 'img','html', 'https', 'org', 'www', 'jpg', 'png', 'net','com','php', 'uid','src', 'ahttp', 'index', 'htm']
pattern = re.compile(r'\b(' + r'|'.join(stopwords_new) + r')\b\s*')
shortword = re.compile(r'\W*\b\w{1,2}\b')
musor = re.compile(r'\d{1}(\w{2})\d{2,4}')

In [195]:
stemmer_ru = RussianStemmer()
stemmer_eng = EnglishStemmer()
stemmer_ger = GermanStemmer()

In [196]:
stop_words = set(stopwords.words(['english', 'russian', 'german']))

In [197]:
my_position_start = 1
my_position_end = 18685
queue = Queue() # очередь ссылок на книги
for i in range(my_position_start, my_position_end):
    queue.put(i)

In [198]:
#split title to words
def split_title(title):
    words = nltk.word_tokenize(title)
    without_extra_words = [stemmer_ru.stem(stemmer_eng.stem(stemmer_ger.stem(word))) for word in words if word not in stop_words]
    return without_extra_words

In [199]:
#split text to words
def split_text(text):
    words = nltk.word_tokenize(text)
    without_extra_words = [stemmer_ru.stem(stemmer_eng.stem(stemmer_ger.stem(word))) for word in words if word not in stop_words]
    return without_extra_words

In [200]:
#returns words which title contains and text contains
def get_content(title, text, n=None):
    title_words = split_title(title)
    text_words = split_text(text)
    if n is not None:
        vectorizer = CountVectorizer().fit(text_words)
        tmp_words_matrix = vectorizer.transform(text_words)
        tmp_words_count = np.sum(tmp_words_matrix, axis=0)
        tmp_words = [(word, tmp_words_count[0, ind]) for word, ind in vectorizer.vocabulary_.items()]
        tmp_words = sorted(tmp_words, reverse=True, key=lambda x: x[1])
        tmp_words = [word[0] for word in tmp_words[:n]]
        doc_words = title_words + tmp_words
    else:
        doc_words = title_words + text_words
    return doc_words

In [203]:
def process_doc(number_of_doc):
    with open('./content/{:d}.dat'.format(number_of_doc), encoding='utf-8') as file:
        text_checker = {}
        html_test = file.read()
        soup = BeautifulSoup(html_test, 'html.parser')
        if soup.title:
            title_name = soup.title.text
            title_name = regex_symbols.sub(" ", title_name)
            title_name = re.sub("\s\s+" , " ", title_name)
        else:
            title_name = ' '
        url = soup.text[:soup.text.index('\n')]
        url = tldextract.extract(url)
        url = url.domain + '.' + url.suffix
        text = soup.text[soup.text.index('\n'):].lower()
        text = regex_symbols.sub(" ", text)
        text = shortword.sub(" ", text)
        text = pattern.sub(" ", text)
        text = musor.sub(' ', text)
        text = re.sub("\s\s+" , " ", text)
        text = re.sub('\xa0|\xad', ' ', text)
        print(text)
        content = get_content(title_name, text, n=25)
        text_checker[number_of_doc] = [url] + content
        return text_checker

In [ ]:
process_doc(25838)

In [ ]:
def process_all_docs(i):
    with gzip.open('data/part_{:05d}.jsonl.gz'.format(i), mode='wb') as f_json:
        f_json = codecs.getwriter('utf8')(f_json)

        while not queue.empty():
            try:
                id_new = queue.get()
                record = process_doc(id_new)
            except Exception as e:
                print(id_new, file=sys.stderr)
                print(e, file=sys.stderr)
            record_str = json.dumps(record, ensure_ascii=False)
            print(record_str, file=f_json)

            # счетчик должен атомарно обновиться
            with lock:
                pbar.update(1)


with Pool(processes=8) as pool, tqdm(total=queue.qsize()) as pbar:
    lock = pbar.get_lock()
    pool.map(process_all_docs, range(pool._processes))

  8%|▊         | 765/9342 [14:58<3:39:55,  1.54s/it] 10114
'NoneType' object has no attribute 'text'
 50%|█████     | 4681/9342 [1:36:40<1:36:11,  1.24s/it]14030
'NoneType' object has no attribute 'text'
 53%|█████▎    | 4934/9342 [1:41:12<2:27:30,  2.01s/it]